# Setep 5: Machine Learning Model

<br>
Clean up any values left from any previous steps

In [26]:
#Quick clean up
for name in dir():
    if not name.startswith('_'): # and name not in ['mbti_FE','mbti_Dataset', 'full_Lem_CV', 'full_Lem_Ngram', 'full_Lem_tfidf']:
        del globals()[name]

Load Dataset and results from previous steps.

In [28]:
#Load information from prevous steps
import pandas as pd
import numpy as np
from scipy import sparse

#mbti_Dataset = pd.read_csv('mbti_Dataset.csv')
#mbti_FE = pd.read_csv('mbti_FE.csv')

#full_Lem_CV = sparse.load_npz('full_Lem_CV.npz')
full_Lem_Ngram = sparse.load_npz('full_Lem_Ngram.npz')
#full_Lem_tfidf = sparse.load_npz('full_Lem_tfidf.npz')


#np.save('full_Lem_CV', full_Lem_CV.toarray())
np.save('d:\full_Lem_Ngram', full_Lem_Ngram.toarray())
#np.save('full_Lem_tfidf', full_Lem_tfidf.toarray())

OSError: [Errno 22] Invalid argument: 'd:\x0cull_Lem_Ngram.npy'

## 5-1: Random Forest Model
### 5-1-1: Random Forest with Holdout test set

In [9]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

#additonal_Features = ['No_Characters', 'No_Words', 'No_Char-Capital', 'No_Words-Capital', 'No_Punctuations', 'No_WordsInQuotes', 'No_Sentences', 'No_UniqueWords', 'No_Stopwords', 'Avg_WordLength', 'Avg_SentLength', 'UniqueWrd_vs_NoWrd', 'Stopwords_vs_NoWrd','Sentiment_Score']

X_Features = full_Lem_tfidf.toarray()
X_Features = pd.DataFrame(X_Features)

def rfClassifier_HoldhoutSet(X_Features, dataset_PD, test_Szie, predict_Lable, is_print = True):
    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie) # 20% of our dataset is test set
    rf = RandomForestClassifier(n_estimators=50, max_depth=20 ,n_jobs=-1)#Max depth of tree is 20
    rf_model = rf.fit(X_train, Y_train)
    #sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10] #List top 10 feature importances
    Y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=predict_Lable, average='binary')
    if is_print:
        print('Results for predicting being ' + predict_Lable ,'Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([precision, recall, fscore, support])


rfClassifier_HoldhoutSet(X_Features, mbti_Dataset['IE'], 0.2, 'I')
rfClassifier_HoldhoutSet(X_Features, mbti_Dataset['NS'], 0.2, 'N')
rfClassifier_HoldhoutSet(X_Features, mbti_Dataset['FT'], 0.2, 'F')
rfClassifier_HoldhoutSet(X_Features, mbti_Dataset['PJ'], 0.2, 'P')

Results for predicting beingI Precision: 0.774 / Recall: 1.0 / Accuracy: 0.774
Results for predicting beingN Precision: 0.856 / Recall: 1.0 / Accuracy: 0.856
Results for predicting beingF Precision: 0.669 / Recall: 0.87 / Accuracy: 0.699
Results for predicting beingP Precision: 0.597 / Recall: 0.995 / Accuracy: 0.598


Explorering our model with grid-search:

In [11]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


def rfClassifier_GridSearch(X_Features, dataset_PD, test_Szie, predict_Lable, n_est, depth, is_print = True,):
    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie) # 20% of our dataset is test set
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth ,n_jobs=-1)#Max depth of tree is 20
    rf_model = rf.fit(X_train, Y_train)
    #sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10] #List top 10 feature importances
    Y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=predict_Lable, average='binary')
    if is_print:
        print('Results for predicting being ' + predict_Lable + ' for '  ,'Est: {} / Depth: {} ----- Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([precision, recall, fscore, support])


X_Features = pd.DataFrame(full_Lem_tfidf.toarray())

for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        rfClassifier_GridSearch(X_Features, mbti_Dataset['IE'], 0.2, 'I',n_est, depth)
        rfClassifier_GridSearch(X_Features, mbti_Dataset['NS'], 0.2, 'N',n_est, depth)
        rfClassifier_GridSearch(X_Features, mbti_Dataset['FT'], 0.2, 'F',n_est, depth)
        rfClassifier_GridSearch(X_Features, mbti_Dataset['PJ'], 0.2, 'P',n_est, depth)


Results for predicting being I for  Est: 10 / Depth: 10 ----- Precision: 0.762 / Recall: 1.0 / Accuracy: 0.762
Results for predicting being N for  Est: 10 / Depth: 10 ----- Precision: 0.858 / Recall: 1.0 / Accuracy: 0.858
Results for predicting being F for  Est: 10 / Depth: 10 ----- Precision: 0.606 / Recall: 0.849 / Accuracy: 0.614
Results for predicting being P for  Est: 10 / Depth: 10 ----- Precision: 0.605 / Recall: 0.988 / Accuracy: 0.603
Results for predicting being I for  Est: 10 / Depth: 20 ----- Precision: 0.774 / Recall: 0.999 / Accuracy: 0.773
Results for predicting being N for  Est: 10 / Depth: 20 ----- Precision: 0.876 / Recall: 1.0 / Accuracy: 0.876
Results for predicting being F for  Est: 10 / Depth: 20 ----- Precision: 0.614 / Recall: 0.775 / Accuracy: 0.625
Results for predicting being P for  Est: 10 / Depth: 20 ----- Precision: 0.619 / Recall: 0.913 / Accuracy: 0.612
Results for predicting being I for  Est: 10 / Depth: 30 ----- Precision: 0.76 / Recall: 0.997 / Accura

Evaluate Random Model

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

rf = RandomForestClassifier()
param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)#cv=5 meand 5 folde validation
gs_fit = gs.fit(full_Lem_tfidf, mbti_Dataset['IE'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.229437,0.028390,0.160970,0.015081,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.771182,0.774640,0.772334,0.768876,0.767147,0.770836,0.002619,1
11,140.384429,10.415539,1.838482,0.198458,None,300,"{'max_depth': None, 'n_estimators': 300}",0.770605,0.770605,0.770605,0.769452,0.770605,0.770375,0.000461,2
7,61.349528,0.319875,1.198994,0.112245,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.770029,0.770605,0.770605,0.769452,0.770605,0.770259,0.000461,3
4,48.727883,0.774539,1.041813,0.022974,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.770605,0.769452,0.771182,0.769452,0.770029,0.770144,0.000672,4
8,121.526988,0.669293,2.227245,0.189642,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.770029,0.770029,0.771182,0.769452,0.770029,0.770144,0.000565,4


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

rf = RandomForestClassifier()
param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)#cv=5 meand 5 folde validation
gs_fit = gs.fit(full_Lem_CV, mbti_Dataset['IE'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
9,5.555741,0.152567,0.145610,0.008649,None,10,"{'max_depth': None, 'n_estimators': 10}",0.786744,0.776369,0.770029,0.756772,0.774640,0.772911,0.009747,1
3,3.167928,0.094107,0.131447,0.005139,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.773487,0.771758,0.772334,0.771758,0.769452,0.771758,0.001314,2
6,4.013665,0.037309,0.136834,0.006226,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.767723,0.775216,0.781556,0.766571,0.765994,0.771412,0.006063,3
0,1.725784,0.039591,0.151995,0.024345,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.770605,0.773487,0.768876,0.769452,0.771182,0.770720,0.001606,4
10,80.572718,0.898699,1.205576,0.079296,None,150,"{'max_depth': None, 'n_estimators': 150}",0.770029,0.771758,0.770605,0.768876,0.770605,0.770375,0.000936,5


In [30]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

Gradient boosting

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


def GBoosting_GridSearch(X_Features, dataset_PD, test_Szie, predict_Lable, n_est, depth, lr ,is_print = True,):
    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie) # 20% of our dataset is test set
    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=depth ,learning_rate=lr)#Max depth of tree is 20
    gb_model = gb.fit(X_train, Y_train)
    #sorted(zip(gb_model.feature_importances_, X_train.columns), reverse=True)[0:10] #List top 10 feature importances
    Y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=predict_Lable, average='binary')
    if is_print:
        print('Results for predicting being ' + predict_Lable + ' for '  ,'Est: {} / Depth: {} / LearningRate : {} ----- Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        lr,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([precision, recall, fscore, support])


X_Features = pd.DataFrame(full_Lem_tfidf.toarray())

for n_est in [50, 100, 150]:
    for depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            GBoosting_GridSearch(X_Features, mbti_Dataset['IE'], 0.2, 'I',n_est, depth, lr)
            GBoosting_GridSearch(X_Features, mbti_Dataset['NS'], 0.2, 'N',n_est, depth, lr)
            GBoosting_GridSearch(X_Features, mbti_Dataset['FT'], 0.2, 'F',n_est, depth, lr)
            GBoosting_GridSearch(X_Features, mbti_Dataset['PJ'], 0.2, 'P',n_est, depth, lr)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

gb = GradientBoostingClassifier()
param = {'n_estimators' : [50, 100, 150],
        'max_depth' : [7, 11, 15],
        'Learning_rate' : [0.1]}

gs = GridSearchCV(gb, param, cv=5)#cv=5 meand 5 folde validation
gs_fit = gs.fit(full_Lem_CV, mbti_Dataset['IE'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]